## <center>Лабораторная работа №10.</center>
## <center>Построение рекомендательной системы видеоконтента с implicit feedback</center>

На базе датасета с фактами покупок абонентами телепередач от компании E-Contenta нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами.

### Вступление. Немного про выбор метода решения (Factorization Machines)

   При решении 9-й лабораторной работы методом ALS на базе user-item-rating информации можно было достичь необходимого результата по RMSE без использования доп.фичей. А так хотелось...но было непонятно как. Метод [Factorization Machines](http://www.algo.uni-konstanz.de/members/rendle/pdf/Rendle2010FM.pdf) предоставляет такую возможность. 
   Ключевыми плюсами этого метода я бы выделил:
 - возможность добавлять неограниченное количество "фичей" за счёт работы с форматом sparse-векторов
 - нелинейную основу алгоритма (когда учитываются не только сами по себе "фичи", но и их взаимодействие между собой)
 - относительно высокую скорость вычислений ($O(kn)$, где $n$ - число "фичей", а $k$ - гиперпараметр модели, который определяет размерность взаимодействующих векторов (как далее будет показано - эта величина была меньше 10)
Есть несколько реализаций представленного в статье алгоритма:
 - библиотека [LibFM](http://www.libfm.org/) от автора статьи
 - библиотека [FastFM](https://github.com/ibayer/fastFM)
 - библиотека [pylibFM](https://github.com/coreylynch/pyFM)

При всём желании воспользоваться решением на Python (pylibFM), сравнение методов было не в его пользу ([Testing implementations of LibFM](http://arogozhnikov.github.io/2016/02/15/TestingLibFM.html). В итоге была выбрана библиотека libFM, которая требует подготовки исходных файлов в заданном формате. После методов онлайн-обучения типа VowpalWabbit это уже знакомая процедура.

   Библиотека [LibFM](http://www.libfm.org/) представляет несколько способов обучения модели (подбора весов коэффициентов $w_{ij}$):
 - MCMC ([Markov chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo))
 - SGD ([Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent))
 - SGDA ([Adaptive Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad))
 - ALS ([Alternating Least Squares](https://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/))
 
В работе ниже были опробованы методы MCMC, SGD и ALS. Максимальную эффективность дал ALS.

Порядок установки и работы с libFM описан в инструкции к библиотеке и в статье [Factorization Machines with libFM](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.442.5724&rep=rep1&type=pdf) от автора алгоритма.
Запускается расчёт следующей командой в командной строке:

In [ ]:
!./libFM -task c -train train.libfm -test test.libfm -method als -dim '1,1,8' -iter 200 \
         -regular ’0,0,15’ -init_stdev 0.1 -out prob9.txt

### Часть 1. Подготовка файла с пользователями и фильмами. Первая проверка работоспособности алгоритма FM на примере libFM.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.simplefilter('ignore')

Загружаем данные из train и test

In [2]:
train = pd.read_csv('data/lab10_train.csv')
test = pd.read_csv('data/lab10_test.csv')

In [3]:
print('Train:', train.shape)
print('Test:', test.shape)

('Train:', (5032624, 3))
('Test:', (2156840, 3))


In [4]:
print('Nums of users:', len(train['user_id'].unique()))
print('Nums of items:', len(train['item_id'].unique()))

('Nums of users:', 1941)
('Nums of items:', 3704)


Сортируем train и test и объединяем в единый датафрейм

In [5]:
train.sort_values(by = ['user_id', 'item_id'], ascending=[True, True], inplace=True)
train.reset_index(drop=True, inplace=True)
test.sort_values(by = ['user_id', 'item_id'], ascending=[True, True], inplace=True)
test.reset_index(drop=True, inplace=True)

In [6]:
full_df = pd.concat([train, test]) 
full_df.reset_index(drop=True, inplace=True)
full_df.reset_index(drop=True, inplace=True)

Формируем словари для перевода поля user_id в формат libFM ("номер непустой колонки:1") и подставляем полученное значение в сводный датафрейм

In [7]:
users = list(set(full_df['user_id']))
users.sort()

user_dic = {}
for idx, usr in enumerate(users):
    user_dic[usr] = idx

full_df['fm_user'] = full_df['user_id'].apply(lambda x: str(user_dic[x])+':1')

Аналогично переводим поле item_id с тем учётом, что нумерация колонок начинается не с 0, а с количества уникальных юзеров

In [8]:
point1=len(user_dic)

In [9]:
films = list(set(full_df['item_id']))
films.sort()

film_dic = {}
for idx, flm in enumerate(films):
    film_dic[flm] = idx+point1

full_df['fm_item'] = full_df['item_id'].apply(lambda x: str(film_dic[x])+':1')

In [10]:
full_df.head()

,user_id,item_id,purchase,fm_user,fm_item
0,1654,326,0.0,0:1,1941:1
1,1654,357,0.0,0:1,1943:1
2,1654,396,0.0,0:1,1944:1
3,1654,400,0.0,0:1,1945:1
4,1654,423,0.0,0:1,1946:1


Проверим - насколько эффективно сработает FM только на данных о том, кто что купил

In [12]:
fm_train_1 = full_df.iloc[0:train.shape[0],2:]
fm_test_1 = full_df.iloc[train.shape[0]:,2:]

fm_test_1['purchase'].fillna(0, inplace=True)
fm_test_1['purchase'] = fm_test_1['purchase'].astype(int)
fm_train_1['purchase'] = fm_train_1['purchase'].astype(int)

fm_train_1.to_csv('train_1.libfm', header = None, index = False, sep = ' ')
fm_test_1.to_csv('test_1.libfm', header = None, index = False, sep = ' ')

In [37]:
prob = pd.read_csv('prob_1.txt', header = None)
out_test = full_df.iloc[train.shape[0]:,0:2]
out_test.reset_index(drop=True, inplace=True)
out_test['purchase'] = prob
out_test.to_csv('../lab10.csv', index=False)

**ROC AUC: 0.922830212573**

Результат хороший, но проверим - сколько даст добавление информации по жанрам и по году выпуска. Задаём смещение номера колонки для следующих данных:

In [13]:
point2 = point1+len(film_dic)

### Часть 2.1. Обработка данных по телепередачам. Формирование данных для дальнейшего кодирования

Загружаем данные по передачам

In [14]:
items_df = pd.read_csv('data/lab10_items.csv', sep = '\t')
items_df.head(2)

,item_id,channel_id,datetime_availability_start,datetime_availability_stop,datetime_show_start,datetime_show_stop,content_type,title,year,genres,region_id
0,65667,NaN,1970-01-01T00:00:00Z,2018-01-01T00:00:00Z,NaN,NaN,1,на пробах только девушки (all girl auditions),2013.0,Эротика,NaN
1,65669,NaN,1970-01-01T00:00:00Z,2018-01-01T00:00:00Z,NaN,NaN,1,скуби ду: эротическая пародия (scooby doo: a x...,2011.0,Эротика,NaN


В таблице много ненужной информации. Оставим следующие поля: item_id, year (год выпуска передачи), genres (жанры передачи)

In [15]:
col2 = [u'item_id', u'year', u'genres']
new_items = items_df[col2]
new_items.sort_values(by = 'item_id', ascending=True, inplace=True)

Также в таблице есть информация по бесплатным телепередачам, которая в первом приближении нам не потребуется. Выделим только платный контент, который есть в train и test

In [16]:
train_items = train['item_id'].unique()
train_items.sort()
tr_items = pd.DataFrame(train_items, columns = ['item_id'])
items = pd.merge(tr_items, new_items, how = 'left', on = 'item_id')

In [17]:
items.head()

,item_id,year,genres
0,326,2012.0,"Ужасы,Триллеры,Драмы,Фантастика,Зарубежные"
1,336,2012.0,"Ужасы,Комедии,Фантастика,Зарубежные"
2,357,2012.0,"Комедии,Мелодрамы,Наши"
3,396,2007.0,"Детективы,Триллеры,Драмы,Фантастика,Зарубежные"
4,400,2010.0,"Фантастика,Боевики,Зарубежные"


### Часть 2.2 Кодирование информации для libFM по свойствам телепередач

Определим множество жанров телепередач

In [18]:
# первым шагом создаём для каждой телепередачи множество её жанров
item_genre_dic = {}
genre = []
for i in tqdm(range(items.shape[0])):
    try:
        item_genre_dic[items.iloc[i,0]] = list(items.iloc[i,2].split(','))
    except:
        AttributeError
        item_genre_dic[items.iloc[i,0]] = ['No']
        continue
gnrs = set()
# вторым шагом создаём итоговое множество всех представленных жанров
for k in item_genre_dic.keys():
    gnrs = gnrs.union(set(item_genre_dic[k]))
gnrs = list(gnrs)
gnrs.sort()

100%|██████████| 3704/3704 [00:02<00:00, 1757.53it/s]


In [19]:
print('Total genres: ' + str(len(gnrs)))

Total genres: 84


Создадим словарь, который каждому жанру будет задавать значение порядкового номера колонки в общем сводном файле. Начинается нумерация со значения point2 = 5645

In [20]:
gnr_dic = {}
for idx, gnr in enumerate(gnrs):
    gnr_dic[gnr] = idx+point2

Для кодирования поля genres в формате libFM с учётом того, что в ячейке может быть указано несколько жанров, создадим вспомогательную функцию, которая на базе строки с жанрами формирует строку с номерами колонок в формате "номер колонки:1" через пробел

In [21]:
def gnr_to_fm(stroka, diction):
    fm_gnr = []
    for gnr in stroka.split(','):
        try:
            fm_gnr.append(str(diction[gnr])+':1')
        except:
            KeyError
            continue
    fm_gnr.sort()
    return ' '.join(fm_gnr)  

Для каждой передачи (фильма) переводим жанр в формат libFM

In [22]:
items['fm_genr'] = 0
for i in tqdm(range(items.shape[0])):
    items.iloc[i,3] = gnr_to_fm(str(items.iloc[i,2]), gnr_dic)

100%|██████████| 3704/3704 [00:03<00:00, 1055.59it/s]


Задаём очередное смещение колонки для данных по году:

In [23]:
point3 = point2+len(gnrs)
point3

5729

Аналогичным образом обрабатываем поле с годом (заменяем пропуски нулями, приводим к int)

In [24]:
#Очищаем данные и создаём список годов выпуска фильмов
items['year'].fillna(0, inplace=True)
items['year'] = items['year'].astype(int)
years = items['year'].unique()
years.sort()
#Создаём словарь для заполнения в дальнейшем колонки в формате libFM
years_dic = {}
for idx, yea in enumerate(years):
    years_dic[yea] = idx+point3

Приводим поле с годом к формату libFM:

In [25]:
items['fm_year'] = 0
for i in tqdm(range(items.shape[0])):
    items.iloc[i,4] = str(years_dic[items.iloc[i,1]])+':1'

100%|██████████| 3704/3704 [00:03<00:00, 1078.41it/s]


In [26]:
items.head()

,item_id,year,genres,fm_genr,fm_year
0,326,2012,"Ужасы,Триллеры,Драмы,Фантастика,Зарубежные",5668:1 5670:1 5717:1 5718:1 5719:1,5804:1
1,336,2012,"Ужасы,Комедии,Фантастика,Зарубежные",5670:1 5674:1 5718:1 5719:1,5804:1
2,357,2012,"Комедии,Мелодрамы,Наши",5674:1 5681:1 5691:1,5804:1
3,396,2007,"Детективы,Триллеры,Драмы,Фантастика,Зарубежные",5658:1 5668:1 5670:1 5717:1 5719:1,5799:1
4,400,2010,"Фантастика,Боевики,Зарубежные",5653:1 5670:1 5719:1,5802:1


Для дальнейшей работы оставим в таблице только поля с номером передачи и закодированными жанром и годом

In [27]:
fm_items = items.drop(['year', 'genres'], axis = 1)

In [28]:
point4 = point3+len(years)
point4

5810

### Часть 3. Формирование файлов train и test для libFM

Объединяем наш исходный датафрейм full_df с пользователями и передачами с датафреймом fm_items со свойствами телепередач. Выделяем из него train и test, предварительно заполнив нулями поле purchase в test

In [29]:
fm_full = pd.merge(full_df, fm_items, how = 'left', on = 'item_id')

fm_train = fm_full.iloc[0:train.shape[0],2:]
fm_test = fm_full.iloc[train.shape[0]:,2:]

fm_test['purchase'].fillna(0, inplace=True)
fm_test['purchase'] = fm_test['purchase'].astype(int)
fm_train['purchase'] = fm_train['purchase'].astype(int)

Записываем данные в файлы в формате libfm:

In [30]:
fm_train.to_csv('train.libfm', header = None, index = False, sep = ' ')
fm_test.to_csv('test.libfm', header = None, index = False, sep = ' ')

In [31]:
#не забываем очистить данные от кавычек, которые достались в наследство при записи строк в файл:
!sed -i 's/"//g' train.libfm 
!sed -i 's/"//g' test.libfm 

### Часть 4. Тренировка модели в libFM

Проверим эффективность предсказательной модели каждым из способов оптимизации:

In [1]:
#MCMC
# !./libFM -task c -train train.libfm -test test.libfm -dim '1,1,8' -out prob7.txt

In [2]:
#SGD
# ./libFM -task c -train train.libfm -test test.libfm -dim '1,1,8' -iter 100 -method sgd -learn_rate 0.01 -regular ’0,0,0.01’ -init_stdev 0.1 -out prob2.txt

In [3]:
#ALS
# !./libFM -task c -train train.libfm -test test.libfm -dim '1,1,8' -iter 200 -method als -regular ’0,0,15’ -init_stdev 0.1 -out prob9.txt

Записываем полученные предсказанные значения вероятности покупки телепередачи в итоговый файл и отправляем на проверку

In [68]:
prob = pd.read_csv('prob2.txt', header = None)
out_test = fm_full.iloc[train.shape[0]:,0:2]
out_test.reset_index(drop=True, inplace=True)
out_test['purchase'] = prob
out_test.to_csv('../lab10.csv', index=False)

 - MCMC: **ROC AUC: 0.90308**
 - SGD: **ROC AUC: 0.89347**
 - ALS: **ROC AUC: 0.93501**

Как видим, полученный результат выше, но не сильно

### Приложение. Обработка данных по активностям пользователей (просмотр бесплатных телепередач)

Загрузим данные из файла lab10_views_programmes.csv, где отражены просмотры бесплатных телепередач всеми пользователями. Файл содержит почти 21 млн.записей по почти 80 000 уникальных пользователей. Для ускорения обработки данных выделим из файла целевую группу пользователей.

In [30]:
views = pd.read_csv('data/lab10_views_programmes.csv')

print('Num of rows:' + str(views.shape[0]))
print('Num of users:' + str(len(views['user_id'].unique())))

Num of rows:20845607
Num of users:79385


In [31]:
tr_users = train['user_id'].unique()
tr_users.sort()
tr_users = pd.DataFrame(tr_users, columns = ['user_id'])
users = pd.merge(tr_users, views, how = 'left', on = 'user_id')
users.head()

,user_id,item_id,ts_start,ts_end,item_type
0,1654,7489015.0,1.493435e+09,1.493435e+09,live
1,1654,7489023.0,1.493444e+09,1.493446e+09,live
2,1654,6617053.0,1.489186e+09,1.489201e+09,live
3,1654,6438693.0,1.487840e+09,1.487840e+09,live
4,1654,6526859.0,1.488705e+09,1.488706e+09,live


In [32]:
users.drop([369911, 676124, 688378], axis =0, inplace = True)

Выделим из этих данных информацию по продолжительности просмотра телепередач, дню недели, перекодируем информацию по полю item_type в формат 1/0.

In [33]:
%%time
#Переведём timestamp-время в стандартный формат:
users['start'] = pd.to_datetime(users['ts_start'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')))
users['finish'] = pd.to_datetime(users['ts_end'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')))
#Вычислим продолжительность просмотра телепередачи:
users['length'] = users['finish'] - users['start']
users['minutes'] = users['length'].apply(lambda x: int(round(x.total_seconds()/60)))
#Вычислим день недели:
users['weekofday'] = [d.isoweekday() for d in users['start']]
#Перекодируем поле item_type:
item_type_dic = {'live':1, 'pvr':0}
users['item_type'] = users['item_type'].map(item_type_dic)

CPU times: user 38.3 s, sys: 4.63 s, total: 43 s
Wall time: 41.9 s


In [34]:
users.head()

,user_id,item_id,ts_start,ts_end,item_type,start,finish,length,minutes,weekofday
0,1654,7489015.0,1.493435e+09,1.493435e+09,1,2017-04-29 06:00:01,2017-04-29 06:10:01,00:10:00,10,6
1,1654,7489023.0,1.493444e+09,1.493446e+09,1,2017-04-29 08:35:01,2017-04-29 09:00:01,00:25:00,25,6
2,1654,6617053.0,1.489186e+09,1.489201e+09,1,2017-03-11 01:49:16,2017-03-11 05:53:54,04:04:38,245,6
3,1654,6438693.0,1.487840e+09,1.487840e+09,1,2017-02-23 11:54:30,2017-02-23 12:00:33,00:06:03,6,4
4,1654,6526859.0,1.488705e+09,1.488706e+09,1,2017-03-05 12:17:32,2017-03-05 12:29:14,00:11:42,12,7


Оставим в таблице только итоговые колонки. Преобразуем данные в новую таблицу, в которой рассчитаем среднее кол-во минут просмотра телепередач в зависмости от дня недели. А также посчитаем, сколько "в среднем" тратится времени на телевизор

In [35]:
col1 = [u'user_id', u'item_type', u'weekofday', u'minutes']
new_views = users[col1]
for col in col1:
    new_views[col] = new_views[col].astype(int)

In [36]:
weeks = new_views.pivot_table(values = ['minutes'], index = ['user_id'], columns =['weekofday'], aggfunc=np.mean)
weeks.reset_index(inplace = True)
for col in weeks.columns:
    weeks[col].fillna(0, inplace = True)
weeks.head()

user_id    minutes                                              \
weekofday                  1          2          3          4          5   
0            1654  15.555556  41.136364  27.280000  35.625000  25.978261   
1          510087  91.754717  77.451220  73.557377  57.606061  71.848101   
2          517612  60.666667  38.600000  55.111111  56.181818  27.142857   
3          520446  76.800000  63.135135  59.033333  65.976190  63.045455   
4          522798  49.572165  69.318627  61.846154  64.309322  61.789700   

                                 
weekofday          6          7  
0          33.111111  27.789474  
1          68.894118  53.265487  
2          71.888889  67.166667  
3          63.516129  51.083333  
4          60.452107  72.521401

In [ ]:
#new_views.groupby(['user_id'])['item_type'].mean()

Нормализуем среднее суммарное время, и рассчитаем распределение времени по дням недели в долях (в сумме = 1)

In [37]:
weeks['sum_min'] = weeks.apply(np.sum, axis =1) - weeks['user_id']
weeks['sum_fit'] = MinMaxScaler().fit_transform(weeks['sum_min'])
col_week = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
for i in range(7):
    weeks[col_week[i]] = weeks['minutes'][i+1]/weeks['sum_min']

Выделим из получившейся таблицы интересующие нас столбцы и строки

In [38]:
new_weeks = weeks.iloc[:,9:]
new_weeks.columns = new_weeks.columns.droplevel(1)
new_weeks['user_id'] = weeks['user_id']
new_weeks.head()

,sum_fit,Mo,Tu,We,Th,Fr,Sa,Su,user_id
0,0.070537,0.075338,0.199231,0.132122,0.172538,0.125817,0.160363,0.134590,1654
1,0.172343,0.185597,0.156664,0.148788,0.116523,0.145331,0.139355,0.107743,510087
2,0.130752,0.161023,0.102453,0.146277,0.149119,0.072043,0.190809,0.178275,517612
3,0.154030,0.173524,0.142649,0.133382,0.149069,0.142447,0.143510,0.115419,520446
4,0.153047,0.112713,0.157611,0.140620,0.146221,0.140492,0.137451,0.164893,522798


Перекодируем данные в формат libFM:

In [39]:
col3 = []
for col in new_weeks.columns:
    col3.append(col+'_fm')
col3.remove('user_id_fm')

fm_weeks = pd.DataFrame()
fm_weeks['user_id'] = new_weeks['user_id']
for idx,col in enumerate(col3):
    fm_weeks[col] = new_weeks[new_weeks.columns[idx]].apply(lambda x: str(idx+point4)+':'+str(x))
    
fm_weeks.head()

,user_id,sum_fit_fm,Mo_fm,Tu_fm,We_fm,Th_fm,Fr_fm,Sa_fm,Su_fm
0,1654,5810:0.0705373704383,5811:0.0753384086812,5812:0.199230954126,5813:0.132122043567,5814:0.172538409167,5815:0.1258174822,5816:0.160363184193,5817:0.134589518065
1,510087,5810:0.17234322999,5811:0.185596623979,5812:0.156664260296,5813:0.148788000206,5814:0.11652251482,5815:0.145330566896,5816:0.139355403929,5817:0.107742629874
2,517612,5810:0.130751510831,5811:0.161022898711,5812:0.102453031158,5813:0.146277212016,5814:0.149119107997,5815:0.0720432121391,5816:0.190809185835,5817:0.178275352144
3,520446,5810:0.15403045725,5811:0.173524195303,5812:0.142649394788,5813:0.133381662275,5814:0.149068559395,5815:0.142446767806,5816:0.143510223686,5817:0.115419196746
4,522798,5810:0.153047375721,5811:0.112712816944,5812:0.157610581964,5813:0.14062033047,5814:0.146220865071,5815:0.140491969718,5816:0.137450670327,5817:0.164892765506


Добавим эти знания о поведении пользователя в сводный датасет

In [40]:
fm_full_2 = pd.merge(fm_full, fm_weeks, how = 'left', on = 'user_id')

fm_train = fm_full_2.iloc[0:train.shape[0],2:]
fm_test = fm_full_2.iloc[train.shape[0]:,2:]

fm_test['purchase'].fillna(0, inplace=True)
fm_test['purchase'] = fm_test['purchase'].astype(int)
fm_train['purchase'] = fm_train['purchase'].astype(int)

In [41]:
fm_train.to_csv('train_2.libfm', header = None, index = False, sep = ' ')
fm_test.to_csv('test_2.libfm', header = None, index = False, sep = ' ')

In [42]:
!sed -i 's/"//g' train_2.libfm 
!sed -i 's/"//g' test_2.libfm 

In [43]:
prob = pd.read_csv('prob_2.txt', header = None)
out_test = fm_full_2.iloc[train.shape[0]:,0:2]
out_test.reset_index(drop=True, inplace=True)
out_test['purchase'] = prob
out_test.to_csv('../lab10.csv', index=False)

На выходе, получаем улучшение модели: **ROC AUC: 0.935881754064**